In [1]:
import pandas as pd 

In [2]:
df = pd.read_excel('tables/violation_rates.xlsx')

In [3]:
df_sum =df.describe()

In [4]:
df_sum.to_excel('tables/summary_statistics.xlsx')

In [7]:
print(df_sum.to_latex(float_format="%.4f"))

\begin{tabular}{lrrr}
\toprule
{} &   RoCoF &     AFV &      AV \\
\midrule
count & 35.0000 & 35.0000 & 35.0000 \\
mean  &  0.1462 &  0.2558 &  0.2947 \\
std   &  0.0066 &  0.0036 &  0.0047 \\
min   &  0.1242 &  0.2470 &  0.2781 \\
25\%   &  0.1431 &  0.2542 &  0.2939 \\
50\%   &  0.1486 &  0.2550 &  0.2951 \\
75\%   &  0.1497 &  0.2572 &  0.2958 \\
max   &  0.1525 &  0.2651 &  0.3034 \\
\bottomrule
\end{tabular}

